In [1]:
# --- Notebook setup ---
%matplotlib inline

import os
import glob
import traceback
import importlib.util
import ipywidgets as widgets
from IPython.display import display, clear_output


In [2]:
#paths to your modules - change these to your actual paths
CWPREPROCESSING_PATH = r"E:\Python\Project\cwpreprocessing.py"
TESTRESULTS_PATH     = r"E:\Python\Project\testresults.py"
STUDPERF_PATH        = r"E:\Python\Project\studentpreformance.py"
UNDERPERF_PATH       = r"E:\Python\Project\underpreforming_student.py"

def import_from_path(module_name: str, file_path: str):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Module not found at: {file_path}")
    spec = importlib.util.spec_from_file_location(module_name, file_path)
    mod = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(mod)  # type: ignore
    return mod

cwprep_mod   = import_from_path("cwpreprocessing", CWPREPROCESSING_PATH)
test_mod     = import_from_path("testresults", TESTRESULTS_PATH)
studperf_mod = import_from_path("studentpreformance", STUDPERF_PATH)
under_mod    = import_from_path("underpreforming_student", UNDERPERF_PATH)

CSVtoSQLite            = cwprep_mod.CSVtoSQLite
TestResultsAnalyzer    = test_mod.TestResultsAnalyzer
StudentPerformance     = studperf_mod.StudentPerformance
UnderperformingStudents= under_mod.UnderperformingStudents

print("Modules loaded.")


Modules loaded.


In [3]:
import sqlite3, pandas as pd

DB = "CWDatabase.db"  # must match your GUI DB path

def norm(s):
    return s.lower().replace(" ", "").replace("_","").replace("-","")

id_keywords = ["researcherid","researcher","studentid","student","candidateid","candidate","userid","user","id"]

conn = sqlite3.connect(DB)
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;", conn)["name"].tolist()
tables = [t for t in tables if "_dfFormattedCleanTest_" in t]
print("Tables:", tables)

found_any = False

for t in tables:
    cols = pd.read_sql(f"PRAGMA table_info('{t}');", conn)["name"].tolist()
    id_col = None
    for c in cols:
        if any(k in norm(c) for k in id_keywords):
            id_col = c
            break

    if id_col:
        found_any = True
        sample = pd.read_sql(
            f"SELECT [{id_col}] AS id FROM [{t}] WHERE [{id_col}] IS NOT NULL LIMIT 15;",
            conn
        )
        print(f"\n Table: {t} | ID column detected: {id_col}")
        print(sample["id"].tolist())

if not found_any:
    print("\nNo ID-like column found in any table.")

conn.close()


Tables: ['Formative_Mock_Test_dfFormattedCleanTest_1', 'Formative_Test_1_dfFormattedCleanTest_2', 'Formative_Test_2_dfFormattedCleanTest_3', 'Formative_Test_3_dfFormattedCleanTest_4', 'Formative_Test_4_dfFormattedCleanTest_5', 'StudentRate_dfFormattedCleanTest_6', 'SumTest_dfFormattedCleanTest_7']

 Table: Formative_Mock_Test_dfFormattedCleanTest_1 | ID column detected: researchid
[34, 73, 15, 156, 135, 12, 145, 153, 69, 21, 2, 101, 155, 80, 95]

 Table: Formative_Test_1_dfFormattedCleanTest_2 | ID column detected: researchid
[156, 155, 154, 153, 152, 150, 149, 148, 147, 146, 145, 143, 142, 141, 140]

 Table: Formative_Test_2_dfFormattedCleanTest_3 | ID column detected: researchid
[21, 2, 18, 17, 147, 24, 32, 19, 101, 77, 135, 96, 155, 130, 149]

 Table: Formative_Test_3_dfFormattedCleanTest_4 | ID column detected: researchid
[110, 119, 45, 120, 4, 73, 156, 149, 25, 24, 80, 51, 10, 131, 154]

 Table: Formative_Test_4_dfFormattedCleanTest_5 | ID column detected: researchid
[126, 62, 135

In [ ]:
import sqlite3, pandas as pd

DB="CWDatabase.db"
conn=sqlite3.connect(DB)

tables=pd.read_sql("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;", conn)["name"].tolist()
formatted=[t for t in tables if "_dfTest_" in t]

print("First formatted table:", formatted[0])
df=pd.read_sql_query(f"SELECT * FROM '{formatted[0]}' LIMIT 5;", conn)
conn.close()

print(df.columns.tolist())
display(df)

First formatted table: Formative_Mock_Test_dfCleanTest_1
['researchid', 'Startedon', 'Completed', 'score', 'Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10']


,researchid,Startedon,Completed,score,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10
0,34,7 November 2018 9:08 AM,7 November 2018 10:55 AM,37.0,5.0,0.0,6.0,7.0,5.0,4.0,10.0,0.0,0.0,0.0
1,73,7 November 2018 9:20 AM,7 November 2018 10:50 AM,94.0,5.0,3.0,6.0,7.0,5.0,4.0,10.0,20.0,20.0,14.0
2,15,7 November 2018 9:20 AM,7 November 2018 10:16 AM,100.0,5.0,3.0,6.0,7.0,5.0,4.0,10.0,20.0,20.0,20.0
3,156,7 November 2018 9:20 AM,7 November 2018 10:48 AM,71.0,5.0,3.0,6.0,7.0,0.0,0.0,10.0,0.0,20.0,20.0
4,135,7 November 2018 9:20 AM,7 November 2018 10:33 AM,100.0,5.0,3.0,6.0,7.0,5.0,4.0,10.0,20.0,20.0,20.0


In [5]:
# ##################
# MAIN WIDGETS UI
###################

out = widgets.Output(layout={"border": "1px solid #ddd", "padding": "10px"})
# DB name and path here, replace if you want something else
db_path = widgets.Text(
    value="CWDatabase.db",
    description="DB path:",
    placeholder="e.g. CWDatabase.db or /path/to/CWDatabase.db",
    layout=widgets.Layout(width="80%")
)

status = widgets.HTML(value="<b>Status:</b> Ready")

def error_code(fn):
    with out:
        clear_output()
        try:
            fn()
        except Exception as e:
            print("Error:")
            print(e)
            print("\n= Traceback =")
            traceback.print_exc()

# TAB 1: CSV -> SQLite 
csv_folder = widgets.Text(
    value="",
    description="CSV folder:",
    placeholder="Paste folder path containing CSVs (optional)",
    layout=widgets.Layout(width="80%")
)

csv_files = widgets.Textarea(
    value="",
    description="CSV files:",
    placeholder="Paste ONE CSV path per line (optional). If provided, these override the folder.",
    layout=widgets.Layout(width="80%", height="110px")
)

convert_btn = widgets.Button(description="Convert CSVs → SQLite",)
show_found  = widgets.Button(description="CSVs found",)

def resolve_csv_list():
    lines = [ln.strip() for ln in csv_files.value.splitlines() if ln.strip()]
    if lines:
        return lines

    folder = csv_folder.value.strip()
    if not folder:
        raise ValueError("Provide a CSV folder")

    if not os.path.isdir(folder):
        raise FileNotFoundError(f"Folder not found: {folder}")

    found = sorted(glob.glob(os.path.join(folder, "*.csv")))
    if not found:
        raise FileNotFoundError(f"No CSV files found in: {folder}")
    return found

def on_preview_csvs(_=None):
    def do():
        paths = resolve_csv_list()
        print(f"Found {len(paths)} CSV file(s):")
        for p in paths:
            print(" -", p)
    error_code(do)

def on_convert(_=None):
    def do():
        paths = resolve_csv_list()
        
        print(f"Converting {len(paths)} CSV files into SQLite DB: CWDatabase.db")

        for p in paths:
            print(f"➡ Converting: {p}")
            conv = CSVtoSQLite(p)
            conv.convert()
        print("\nDatabase created/updated: CWDatabase.db")
        status.value = "<b>Status:</b> Import complete. Refresh table list in analysis tabs if needed."
    error_code(do)

show_found.on_click(on_preview_csvs)
convert_btn.on_click(on_convert)

tab1 = widgets.VBox([
    widgets.HTML("<h3>1) Import CSV files into SQLite</h3>"),
    widgets.HTML("Provide a <b>folder</b>"),
    csv_folder,
    csv_files,
    widgets.HBox([show_found, convert_btn]),
])

#  TAB 2: Student results across ALL tables
student_id_all = widgets.Text(value="", description="Student ID:", placeholder="e.g. 12345", layout=widgets.Layout(width="50%"))
fetch_all_btn = widgets.Button(description="Fetch + Plot Student Results")

def on_fetch_all(_=None):
    def do():
        sid = student_id_all.value.strip()
        if not sid:
            raise ValueError("Enter a student ID.")
        df = TestResultsAnalyzer.get_student_scores(db_path.value.strip(), sid)
        if df is None or df.empty:
            print("No rows returned for this student.")
            return
        display(df)
        TestResultsAnalyzer.plot_student_scores(df, sid)
        print("Done.")
    error_code(do)

fetch_all_btn.on_click(on_fetch_all)

tab2 = widgets.VBox([
    widgets.HTML("<h3>2) View all assessment results for one student</h3>"),
    db_path,
    student_id_all,
    fetch_all_btn,
])

# TAB 3: Per-question performance 
sp = None  # will be created when refreshed

refresh_tables_btn = widgets.Button(description="Refresh Table List", icon="refresh")
table_dropdown = widgets.Dropdown(options=[], description="Table:", layout=widgets.Layout(width="80%"))
student_id_q = widgets.Text(value="", description="Student ID:", placeholder="e.g. 12345", layout=widgets.Layout(width="50%"))
analyse_btn = widgets.Button(description="Analyse + Plot")

def refresh_tables(_=None):
    def do():
        global sp
        sp = StudentPerformance(db_path.value.strip())
        tables = sp.list_tables()
        if not tables:
            table_dropdown.options = []
            table_dropdown.value = None
            print("No tables found. Import CSVs first.")
            return
        default = sp.pick_default_test_table(tables)
        table_dropdown.options = tables
        table_dropdown.value = default if default in tables else tables[0]
        print("list refreshed.")
        print("Selected:", table_dropdown.value)
    error_code(do)

def on_analyse_question_level(_=None):
    def do():
        if sp is None:
            raise ValueError("Click 'Refresh Table List' first.")
        sid = student_id_q.value.strip()
        if not sid:
            raise ValueError("Enter a student ID.")
        table = table_dropdown.value
        if not table:
            raise ValueError("No table selected.")
        result = sp.analyse(sid, table)
        display(result)
        sp.plot(result, sid, table)
        print("Done.")
    error_code(do)

refresh_tables_btn.on_click(refresh_tables)
analyse_btn.on_click(on_analyse_question_level)

tab3 = widgets.VBox([
    widgets.HTML("<h3>3) Student performance</h3>"),
    db_path,
    widgets.HBox([refresh_tables_btn, widgets.HTML("<span style='color:#666'>Refresh after importing CSVs or changing DB path.</span>")]),
    table_dropdown,
    student_id_q,
    analyse_btn
])

# TAB 4: Underperforming students report 
threshold = widgets.FloatSlider(
    description="Threshold:",
    readout_format=".0f",
    layout=widgets.Layout(width="80%")
)

summative_table = widgets.Text(
    value="",
    description="Summative table:",
    placeholder="Leave blank for auto-detect.",
    layout=widgets.Layout(width="80%")
)

build_btn = widgets.Button(description="Build Report + Plot")

def on_build_report(_=None):
    def do():
        ups = UnderperformingStudents(db_path.value.strip())
        summ = summative_table.value.strip() or None
        report, used_summ = ups.build_report(summative_table=summ, threshold=float(threshold.value))
        display(report)
        ups.plot_underperformers(report, used_summ, float(threshold.value))
        print(f"Summative table used: {used_summ}")
    error_code(do)

build_btn.on_click(on_build_report)

tab4 = widgets.VBox([
    widgets.HTML("<h3>4) Underperforming students</h3>"),
    db_path,
    summative_table,
    threshold,
    build_btn
])

#  Assemble 
tabs = widgets.Tab(children=[tab1, tab2, tab3, tab4, out])
tabs.set_title(0, "CSV → SQLite")
tabs.set_title(1, "Student Results")
tabs.set_title(2, "Per-Question Analysis")
tabs.set_title(3, "Underperformers")
tabs.set_title(4, "Output / Logs")

display(widgets.VBox([
    widgets.HTML("<h2>Menu Toolkit</h2>"),
    status,
    tabs
]))

